## Install the libraries and prepare the API model

In [ ]:
# import the inference-sdk
%pip install inference_sdk
from google.colab import drive
from inference_sdk import InferenceHTTPClient

# initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="WFIbKZhlG2t0OkPoFFDm"
)

## Sample testing images are stored in the google drive and we are mounting them

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path in Google Drive
base_dir = '/content/drive/MyDrive/Colab Notebooks/Ukraine locations/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Run the model on each image from location folder

In [ ]:
from PIL import Image
import os
from datetime import datetime

# Cost estimates for each damage category (min, max) from the database
cost_estimates = {
    "broken_window": (200, 800),
    "roof_damage": (1500, 9000),
    "crack_damage": (500, 5000),
    "minor_damage": (100, 500),
    "major_damage": (5000, 20000)
}

# Define the base directory with subfolders representing building locations
base_dir = "/content/drive/MyDrive/Colab Notebooks/Ukraine locations/"  # Google Drive path

# Iterate over the folders in the base directory (each representing a location)
for location_folder in os.listdir(base_dir):
    location_path = os.path.join(base_dir, location_folder)
    if not os.path.isdir(location_path):
        continue

    # Initialize damage counts and cost totals
    damage_count = {key: 0 for key in cost_estimates.keys()}
    min_total_cost, max_total_cost = 0, 0

    # Process each image in the location folder
    for img_name in os.listdir(location_path):
        if img_name.endswith(".png") or img_name.endswith(".jpg"):
            img_path = os.path.join(location_path, img_name)

            # Open the image
            img = Image.open(img_path)

            # Convert RGBA to RGB if necessary
            if img.mode == 'RGBA':
                img = img.convert('RGB')

            # Run inference on the image (replace with your inference model code)
            result = CLIENT.infer(img_path, model_id="building-damage-dlnea/2")

            # Process predictions
            for pred in result["predictions"]:
                damage_type = pred["class"]
                if damage_type in damage_count:
                    damage_count[damage_type] += 1
                    # Add to the cost totals
                    min_total_cost += cost_estimates[damage_type][0]
                    max_total_cost += cost_estimates[damage_type][1]

    # Print Report for the current location
    print(f"\nBuilding Damage Assessment Report for {location_folder}")

    # Print Header
    print("===============================================")
    print(f"Location: {location_folder}")
    report_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Report Generated on: {report_date}")
    print("===============================================")

    # Print Damage Summary Table
    print("| Category       | Count | Estimated Cost (if available) |")
    print("-----------------------------------------------")
    for damage_type, count in damage_count.items():
        if count > 0:
            min_cost, max_cost = cost_estimates[damage_type]
            estimated_cost = f"${min_cost * count} - ${max_cost * count}"
            print(f"| {damage_type.ljust(15)} | {str(count).ljust(5)} | {estimated_cost.ljust(30)} |")

    # Print Total Cost Summary
    print("-----------------------------------------------")
    print(f"Total Estimated Cost: ${min_total_cost} - ${max_total_cost}")
    print("===============================================\n")

print("Reports printed successfully!")



Building Damage Assessment Report for 50.4501_30.5234
Location: 50.4501_30.5234
Report Generated on: 2024-11-16 14:50:53
| Category       | Count | Estimated Cost (if available) |
-----------------------------------------------
| broken_window   | 6     | $1200 - $4800                  |
| roof_damage     | 1     | $1500 - $9000                  |
-----------------------------------------------
Total Estimated Cost: $2700 - $13800


Building Damage Assessment Report for 47.8388_35.1396
Location: 47.8388_35.1396
Report Generated on: 2024-11-16 14:50:55
| Category       | Count | Estimated Cost (if available) |
-----------------------------------------------
| broken_window   | 4     | $800 - $3200                   |
| roof_damage     | 1     | $1500 - $9000                  |
| crack_damage    | 3     | $1500 - $15000                 |
| minor_damage    | 2     | $200 - $1000                   |
-----------------------------------------------
Total Estimated Cost: $4000 - $28200

Repo